In [3]:
import torch
import torch.nn as nn
import time
import dgl
import numpy as np
import time
from torch.utils import data
import matplotlib.pyplot as plt
import random
from ode_nn import GCN, GAT, Dataset_graph, train_epoch_graph, eval_epoch_graph, get_lr
import warnings
warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="4"

Using backend: pytorch


In [2]:
input_length = 14
mid = 14
output_length = 7
batch_size = 2

###################################
train_direc = '/home/ec2-user/SageMaker/efs/Danielle/Ray/Data/Latest_US_Covid_Data_0726/graph/train/sample_'
test_direc = '/home/ec2-user/SageMaker/efs/Danielle/Ray/Data/Latest_US_Covid_Data_0726/graph/test/sample_'
train_indices = list(range(50))
valid_indices = list(range(50, 56))
test_indices = list(range(3))

###################################
train_set = Dataset_graph(train_indices, input_length, mid, output_length, train_direc)#, entire_target = True
valid_set = Dataset_graph(valid_indices, input_length, mid, output_length, train_direc)#, entire_target = True
test_set = Dataset_graph(test_indices, input_length, mid, output_length, test_direc)#, entire_target = True
train_loader = data.DataLoader(train_set, batch_size = batch_size, shuffle = True)
valid_loader = data.DataLoader(valid_set, batch_size = batch_size, shuffle = False)
test_loader = data.DataLoader(test_set, batch_size = batch_size, shuffle = False)

In [6]:
graph = torch.load("us_graph.pt")
G = dgl.DGLGraph().to(device)
G.add_nodes(56)
for i in range(56):
    for j in range(56):
        if graph[i,j] == 1:
            G.add_edge(i,j)

In [48]:
#model = GCN(in_dim = 42, out_dim = 3, hidden_dim = 64, num_layer = 5).to(device)
model = GAT(in_dim = 42, out_dim = 3, hidden_dim = 32, num_heads = 4, num_layer = 6).to(device)

In [ ]:
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 1, gamma=0.9)
loss_fun = nn.MSELoss()
sum(p.numel() for p in model.parameters() if p.requires_grad) 
train_rmse = []
valid_rmse = []
test_rmse = []
min_rmse = 1
for i in range(1, 100):
    start = time.time()
    scheduler.step()
    model.train()
    train_rmse.append(train_epoch_graph(model, train_loader, optimizer, loss_fun, G)[-1])
    model.eval()
    preds, trues, rmse = eval_epoch_graph(model, valid_loader, loss_fun, G)
    valid_rmse.append(rmse)
    if valid_rmse[-1] < min_rmse:
        min_rmse = valid_rmse[-1] 
        best_model = model 
        #torch.save(best_model, "lstm_us.pth")
    end = time.time()
    if (len(train_rmse) > 30 and np.mean(valid_rmse[-5:]) >= np.mean(valid_rmse[-10:-5])):
            break
    print("Epoch " + str(i) + ": ", "train rmse:", train_rmse[-1], "valid rmse:",valid_rmse[-1], 
              "time:",round((end-start)/60,3), "Learning rate:", format(get_lr(optimizer), "5.2e"))
idx = 3
preds, trues, rmse = eval_epoch_graph(best_model, test_loader, loss_fun, G)
torch.save({"preds": preds,
            "trues": trues,
            "rmse": np.sqrt(np.mean((preds - trues[:,:,-7:])**2))}, 
            "GAT_" + str(idx) + ".pt")